In [1]:
import pandas as pd
import ast

In [24]:
# outlier 제외 maindata 수정 : maindata_var1.2.1
maindata = pd.read_csv('/content/drive/MyDrive/EPOCH/data/maindata_ver1.2.csv')
maindata = maindata[maindata['id'] != 135397]

maindata.to_csv('/content/drive/MyDrive/EPOCH/data/maindata_var1.2.1.csv', index=False)
metadata = pd.read_csv('/content/drive/MyDrive/EPOCH/data/Copy of movies_metadata.csv')

<ipython-input-24-1a83bfb1aa01>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('/content/drive/MyDrive/EPOCH/data/Copy of movies_metadata.csv')


metadata set에서 `production_companies` 열에서 `company_name`, `company_id`, `id` 뽑아내기

In [40]:
import ast
import pandas as pd

# Define a function to extract the list of production_company's names and ids from the production_companies column
def extract_production_company_info(production_company_data):
    if pd.isna(production_company_data):  # Check if the data is NaN
        return pd.Series([[], []])
    try:
        production_company_list = ast.literal_eval(production_company_data)
        if isinstance(production_company_list, list) and len(production_company_list) > 0:
            company_names = [company.get('name') for company in production_company_list]
            company_ids = [company.get('id') for company in production_company_list]
            return pd.Series([company_names, company_ids])
        else:
            return pd.Series([[], []])
    except (ValueError, SyntaxError):  # Catch exceptions related to malformed data
        return pd.Series([[], []])

# Apply the function to extract production company information
metadata[['company_name', 'company_id']] = metadata['production_companies'].apply(extract_production_company_info)

# Create the final DataFrame with company names, company ids, and movie id
production_company_df = metadata[['company_name', 'company_id', 'id']]
production_company_df

# Display the result
# import ace_tools as tools; tools.display_dataframe_to_user(name="Production Company Information", dataframe=production_company_df)

production_company_df.to_csv('/content/drive/MyDrive/EPOCH/data/company.csv', index=False)

`company_name`, `company_id` 컬럼을 `id` 기준으로 maindata_var1.2.1 과 merge

In [38]:
production_company_df['id'] = pd.to_numeric(production_company_df['id'], errors='coerce').astype('Int64')

maindata_new = pd.merge(maindata, production_company_df, on='id', how='inner')

maindata_new.to_csv('/content/drive/MyDrive/EPOCH/data/maindata_var1.2.2.csv', index=False)

<ipython-input-38-31eb9a7bf080>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  production_company_df['id'] = pd.to_numeric(production_company_df['id'], errors='coerce').astype('Int64')
